# Part 2

## Testing the model through webcam feed

In [1]:
import os
import cv2
import numpy as np

In [2]:
from keras.models import load_model

Using TensorFlow backend.


In [3]:
model = load_model('model2.h5')  # loading the model

In [4]:
img_size = 80 # providing same image size on which the model was trained

In [5]:
# checking for a sample image

img = cv2.imread('mask5.jpg')
resized = cv2.resize(img,(img_size,img_size))
normalized = resized/255.0
reshaped = np.reshape(normalized,(1,img_size,img_size,3))
#reshaped = np.reshape(normalized,(1,img_size,img_size,1))  # (in case of grayscale image)
result = model.predict(reshaped)
label = np.argmax(result,axis=1)[0]

print(result)
label = result[0][1] >0.5
print(int(label))

[[9.9917525e-01 8.2469458e-04]]
0


In [6]:
labels_dict={0:'MASK', 1:'NO MASK'}
color_dict={0:(0,255,0),1:(0,0,255)}  # green for with mask, red for without mask

face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')  # detecting faces
source=cv2.VideoCapture(0)

### Detecting faces as with or without mask

In [7]:
while(True):

    ret, img = source.read()
    
    faces = face_cascade.detectMultiScale(img, 1.3, 5) 

    for x,y,w,h in faces:

        face_img = img[y:y+w,x:x+w]  #cropped face image
        
        resized = cv2.resize(face_img, (img_size, img_size))  #resizing image
        
        #resized = cv2.cvtColor(resized,cv2.COLOR_BGR2GRAY)  #converting image in gray scale
        
        normalized = resized/255.0  #normalizing between 0 and 1
        
        reshaped = np.reshape(normalized, (1,img_size,img_size,3))  #converting in 4d array
        
        result=model.predict(reshaped)

        label = result[0][1] >0.5
        label = int(label)
      
        cv2.rectangle(img,(x,y),(x+w,y+h),color_dict[label],2)
        cv2.rectangle(img,(x,y-40),(x+w,y),color_dict[label],-1)
        cv2.putText(img, labels_dict[label] + str(round(result[0][label]*100,2)) + '%', (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(0,0,0),2)
        
        
    cv2.imshow('LIVE Video Feed',img)
    key=cv2.waitKey(1)
    
    if(key==27):  # press esc key to stop
        break
        
cv2.destroyAllWindows()
source.release()